# Battle of the neighbourhoods

## Introduction Problem

Finding Neighborhoods that have facilities that students would like to have. Let's say I am a property developer for student accommodation. I am interested in finding neighbourhoods that are near universities and located near the facilities that students would desire (gyms,  convenience shops, nightclubs) whilst being affordable. So I will analyse areas surrounding universities, to find areas that would provide me with maximal profit whilst keeping being appealing to students in the area. 

King's College which is in London will be the university investigated. London is known to have some of the highest rents in the world, so obtaining affordable yet characteristic accommodation for students is a difficult problem that could result in large financial gains as the ordinary student rent range between £135 - £210 per week [1]. However, due to the extraordinary prices of properties in London, the rent will need to be compared to the values of properties, to determine the yield of the investments.

While the income yield is a crucial measure of investment properties, the focus of this analysis will be on finding neighbourhoods with appealing amenities. There is a saying that is often preached within real estate agencies: 'location, location, location,' and in this report, the aims will be to find the right location, as that is all that matters. 

With the soaring cost of living in London, It is no surprise that 44% of students struggle to pay their rent each month [] and 31 % finding their studies affected. Combined with increasing tuition fees, the crippling student debt crisis is a real problem in London and the rest of the UK. How can this keep going on? Now while attempts are being made to control rents through legislation []. However, this option is lengthly and unlikely to be successful, largely due to political opposition. So, it is our responsibility to seek more desirable housing to reduce students cost, to help the next generation with the continual goal of learning and improving themselves. As it is today's student that will become tomorrows inventors, business owners and professors. 

\
We do not have the perfect conditions for students to find affordable places to live in, but we do have access to vast amounts of data relating to the location of venues in London (using the Foursquare API), the average rental prices in each area (using data from Spare room) and average house prices (using HM land registry).

From the data, we can extract the ideal areas that students would live to live in, by looking at the number of and types of venues in each area. To determine the optimal location, we must first find areas that are within a 30-minute commute of the university (King's College London) and have at least 3 of the following, within a 500-meter range: gym, coffee shop, nightclub, convenience shop. Finding budget areas where these amenities exist could prove to be valuable for students finding affordable accommodation and investors finding high yield properties.

To find ideal locations for students, areas will be clustered by venue and clusters will be compared to find the ones with the most ideal properties. Clusters with the most ideal areas will be extracted and rental prices will be compared to identify the areas where accommodation is affordable for students.